In [15]:
import pandas as pd

df = pd.read_csv("E:/Data Science/EcoPackAI/data/processed/materials_featured.csv")

print(df.shape)
print(df.columns)

(22, 19)
Index(['material_id', 'material_type', 'strength_mpa', 'weight_capacity',
       'co2_emission_per_kg', 'biodegradability_score', 'recyclability_pct',
       'cost_inr_per_kg', 'material_category', 'strength_level', 'co2_norm',
       'cost_norm', 'strength_norm', 'emission_score', 'recyclability_index',
       'co2_impact_index', 'cost_score', 'cost_efficiency_index',
       'material_suitability_score'],
      dtype='object')


In [16]:

strength_mapping = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

df["strength_encoded"] = df["strength_level"].map(strength_mapping)

In [ ]:
X = df[[
    "strength_encoded",          
    "weight_capacity",           
    "biodegradability_score",    
    "recyclability_pct",         
    "cost_efficiency_index"      
]]

In [ ]:
# Cost prediction target
y_cost = df["cost_inr_per_kg"]

# CO2 impact prediction target
y_co2 = df["co2_impact_index"]

print(X.shape)
print(y_cost.shape)
print(y_co2.shape)

(22, 5)
(22,)
(22,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_cost_train, y_cost_test = train_test_split(
    X,
    y_cost,
    test_size=0.2,
    random_state=42
)

print(X_train.shape)
print(X_test.shape)

(17, 5)
(5, 5)


In [8]:
from sklearn.ensemble import RandomForestRegressor

cost_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

cost_model.fit(X_train, y_cost_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

cost_predictions = cost_model.predict(X_test)

print("COST MODEL EVALUATION")
print("MAE:", mean_absolute_error(y_cost_test, cost_predictions))
print("RMSE:", np.sqrt(mean_squared_error(y_cost_test, cost_predictions)))
print("R2:", r2_score(y_cost_test, cost_predictions))

COST MODEL EVALUATION
MAE: 10.554000000000006
RMSE: 14.057496932242238
R2: 0.981402859025033


In [10]:
pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 16.2 MB/s eta 0:00:05
    --------------------------------------- 1.3/72.0 MB 16.0 MB/s eta 0:00:05
   - -------------------------------------- 2.2/72.0 MB 17.3 MB/s eta 0:00:05
   - -------------------------------------- 3.1/72.0 MB 18.3 MB/s eta 0:00:04
   -- ------------------------------------- 4.1/72.0 MB 19.0 MB/s eta 0:00:04
   -- ------------------------------------- 5.0/72.0 MB 18.8 MB/s eta 0:00:04
   --- ------------------------------------ 5.9/72.0 MB 19.0 MB/s eta 0:00:04
   --- ------------------------------------ 6.8/72.0 MB 18.9 MB/s eta 0:00:04
   ---- ----------------------------------- 7.9/72.0 MB 19.3 MB/s eta 0:00:04
   ---- ----------------------------------- 8.9/72.0 MB 19.7 MB/s eta 0:00:04
   ----- ---------------------------------- 10.1/72.0 MB 20.1 MB/s eta 0:00:04
   ------ --------------------------------- 10.8/72.0 MB 20.5 MB/s eta 


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from xgboost import XGBRegressor

co2_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

co2_model.fit(X_train, y_co2.loc[y_cost_train.index])

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

co2_predictions = co2_model.predict(X_test)

print("CO2 MODEL EVALUATION")
print(
    "MAE:",
    mean_absolute_error(y_co2.loc[y_cost_test.index], co2_predictions)
)
print(
    "RMSE:",
    np.sqrt(
        mean_squared_error(
            y_co2.loc[y_cost_test.index],
            co2_predictions
        )
    )
)
print(
    "R2:",
    r2_score(y_co2.loc[y_cost_test.index], co2_predictions)
)

CO2 MODEL EVALUATION
MAE: 0.10224292483246118
RMSE: 0.19359427242000016
R2: 0.2509337809832177


In [ ]:
df_test = df.loc[X_test.index].copy()

df_test["predicted_cost"] = cost_predictions
df_test["predicted_co2"] = co2_predictions

df_test["final_score"] = (
    0.5 * df_test["predicted_cost"].rank(ascending=True) +
    0.5 * df_test["predicted_co2"].rank(ascending=True)
)

df_test_sorted = df_test.sort_values("final_score")

df_test_sorted.head()

,material_id,material_type,strength_mpa,weight_capacity,co2_emission_per_kg,biodegradability_score,recyclability_pct,cost_inr_per_kg,material_category,strength_level,...,emission_score,recyclability_index,co2_impact_index,cost_score,cost_efficiency_index,material_suitability_score,strength_encoded,predicted_cost,predicted_co2,final_score
0,MAT001,HDPE Plastic,26,0.95,1.9,0,90,105,Plastic,Medium,...,0.859649,0.90,0.699825,0.791457,0.494914,0.583151,2,112.51,0.663069,2.0
1,MAT002,LDPE Plastic,12,0.92,2.0,0,70,110,Plastic,Low,...,0.850877,0.70,0.635439,0.778894,0.476154,0.524806,1,118.12,0.593262,2.0
8,MAT009,Corrugated Cardboard,5,0.69,0.9,1,85,45,Paper,Low,...,0.947368,0.85,0.928684,0.942211,0.568533,0.702696,1,53.16,0.931258,3.0
13,MAT014,Aluminum (Primary),95,2.70,11.7,0,95,255,Metal,High,...,0.000000,0.95,0.285000,0.414573,0.324094,0.355521,3,283.26,0.714252,3.5
15,MAT016,PLA Bioplastic,60,1.24,1.6,1,70,320,Biopolymer,High,...,0.885965,0.70,0.852982,0.251256,0.198048,0.534983,3,320.72,0.852525,4.5


In [17]:
df.columns

Index(['material_id', 'material_type', 'strength_mpa', 'weight_capacity',
       'co2_emission_per_kg', 'biodegradability_score', 'recyclability_pct',
       'cost_inr_per_kg', 'material_category', 'strength_level', 'co2_norm',
       'cost_norm', 'strength_norm', 'emission_score', 'recyclability_index',
       'co2_impact_index', 'cost_score', 'cost_efficiency_index',
       'material_suitability_score', 'strength_encoded'],
      dtype='object')

In [ ]:
import joblib

joblib.dump(cost_model, "models/cost_model.pkl")
joblib.dump(co2_model, "models/co2_model.pkl")

print("Models saved successfully.")